# A simple code-graded evaluation

このレッスンでは、次のレッスンでより現実的なプロンプトを扱う前に、まず **とてもシンプルなコード採点（code-graded）の評価** を見ていきます。以下の図のプロセスに沿って進めます：

![process.png](../images/process.png)

大まかな手順は次のとおりです。
1. 評価用テストセットを定義する
2. 初期プロンプト案を書く
3. 評価プロセスに通してスコアを得る
4. 評価結果に基づいてプロンプトを修正する
5. 修正したプロンプトを再度評価し、スコアが改善することを期待する

では実際にこの流れをやってみましょう！

---


## Our input data

ここでは、Claude に「動物の脚の本数」を正しく答えさせる評価を採点します。今後のレッスンではより複雑で現実的なプロンプト／評価を扱いますが、ここでは評価プロセス自体に集中するため、意図的に単純な題材にしています。

最初のステップは、入力と対応するゴールデンアンサーを含む評価データセットを書くことです。以下のような辞書のリストを使い、各辞書に `animal_statement` と `golden_answer` を持たせます：


In [2]:
eval_data = [
    {"animal_statement": "The animal is a human.", "golden_answer": "2"},
    {"animal_statement": "The animal is a snake.", "golden_answer": "0"},
    {"animal_statement": "The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.", "golden_answer": "5"},
    {"animal_statement": "The animal is a dog.", "golden_answer": "4"},
    {"animal_statement": "The animal is a cat with two extra legs.", "golden_answer": "6"},
    {"animal_statement": "The animal is an elephant.", "golden_answer": "4"},
    {"animal_statement": "The animal is a bird.", "golden_answer": "2"},
    {"animal_statement": "The animal is a fish.", "golden_answer": "0"},
    {"animal_statement": "The animal is a spider with two extra legs", "golden_answer": "10"},
    {"animal_statement": "The animal is an octopus.", "golden_answer": "8"},
    {"animal_statement": "The animal is an octopus that lost two legs and then regrew three legs.", "golden_answer": "9"},
    {"animal_statement": "The animal is a two-headed, eight-legged mythical creature.", "golden_answer": "8"},
]

評価用の質問の中には、次のように少しひねったものもあることに注意してください：

> The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.

これは後で重要になります！

---


## Our initial prompt

次に、初期プロンプトを定義します。以下の関数は、1つの動物ステートメントを受け取り、最初のプロンプト案を含む `messages` リスト（Messages API 形式）を返します：


In [3]:
def build_input_prompt(animal_statement):
    user_content = f"""You will be provided a statement about an animal and your job is to determine how many legs that animal has.
    
    Here is the animal statement.
    <animal_statement>{animal_statement}</animal_statement>
    
    How many legs does the animal have? Please respond with a number"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

まずは `eval` データセットの先頭要素で、簡単に試してみましょう：


In [4]:
build_input_prompt(eval_data[0]['animal_statement'])

[{'role': 'user',
  'content': 'You will be provided a statement about an animal and your job is to determine how many legs that animal has.\n\n    Here is the animal statement.\n    <animal_statement>The animal is a human.</animal_statement>\n\n    How many legs does the animal have? Please respond with a number'}]

次に、`messages` のリストを受け取り、Anthropic API に送って応答を得るシンプルな関数を書きます：


In [5]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

#MODEL_NAME = "claude-3-haiku-20240307"
MODEL_NAME = "claude-sonnet-4-5-20250929"

def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=200,
        messages=messages
    )
    return response.content[0].text

`eval_data` の最初のエントリを使って試してみます。ここには次の動物ステートメントが入っています：

```
'The animal is a human.'
```


In [6]:
full_prompt = build_input_prompt(eval_data[0]['animal_statement'])
get_completion(full_prompt)

'2'

応答として `2` が返ってきました。見た目のチェック（eyeball test）も通りますね。人間の脚は一般に2本です。

次は、`eval_data` の全12件すべてで評価を実行し、スコアを出します。

---


## Writing the eval logic

まず `eval_data` の各入力に対してプロンプトテンプレートを埋め込み、完成したプロンプトをモデルに渡し、返ってきた出力をすべて収集します：


In [7]:

outputs = [get_completion(build_input_prompt(question['animal_statement'])) for question in eval_data]


まずは返ってきた結果をざっと見てみましょう：


In [8]:
outputs

['2',
 '0',
 'A fox normally has 4 legs.\n\nAccording to the statement:\n- The fox lost a leg: 4 - 1 = 3 legs\n- Then grew back the lost leg: 3 + 1 = 4 legs\n- Plus grew an extra leg: 4 + 1 = 5 legs\n\n**5**',
 '4',
 'A normal cat has 4 legs.\n\nWith two extra legs, the animal has: 4 + 2 = **6**',
 '4',
 '2',
 '0',
 'A spider normally has 8 legs.\n\nWith two extra legs, the animal would have: 8 + 2 = **10**',
 '0\n\nAn octopus has 8 arms, not legs.',
 'An octopus normally has 8 legs.\n\nIf it lost 2 legs: 8 - 2 = 6 legs\n\nIf it then regrew 3 legs: 6 + 3 = 9 legs\n\n**9**',
 '8']

すでに、プロンプトの改善が必要だと分かります。数値だけではなく、数値以外の文章が混ざった出力が出ています。

各出力を対応するゴールデンアンサーと並べて、もう少し詳しく見てみましょう。


In [9]:
for output, question in zip(outputs, eval_data):
    print(f"Animal Statement: {question['animal_statement']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

Animal Statement: The animal is a human.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a snake.
Golden Answer: 0
Output: 0

Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: A fox normally has 4 legs.

According to the statement:
- The fox lost a leg: 4 - 1 = 3 legs
- Then grew back the lost leg: 3 + 1 = 4 legs
- Plus grew an extra leg: 4 + 1 = 5 legs

**5**

Animal Statement: The animal is a dog.
Golden Answer: 4
Output: 4

Animal Statement: The animal is a cat with two extra legs.
Golden Answer: 6
Output: A normal cat has 4 legs.

With two extra legs, the animal has: 4 + 2 = **6**

Animal Statement: The animal is an elephant.
Golden Answer: 4
Output: 4

Animal Statement: The animal is a bird.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a fish.
Golden Answer: 0
Output: 0

Animal Statement: The animal is a spider with two extra legs
Golden Answer: 10
Output

このデータセットは小さいので目視でも問題点を探せますが、ここでは体系的に採点してみましょう：


In [10]:
def grade_completion(statement, output, golden_answer):
    # 出力とゴールデンアンサーが一致しているかどうかを返す。不一致の場合は内容を表示する
    if output == golden_answer:
        return 1
    else:
        print(f"Animal Statement: {statement}\nGolden Answer: {golden_answer}\nOutput: {output}\n")
        return 0

grades = [grade_completion(question['animal_statement'], output, question['golden_answer']) for output, question in zip(outputs, eval_data)]
print(f"Score: {sum(grades)/len(grades)*100}%")

Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: A fox normally has 4 legs.

According to the statement:
- The fox lost a leg: 4 - 1 = 3 legs
- Then grew back the lost leg: 3 + 1 = 4 legs
- Plus grew an extra leg: 4 + 1 = 5 legs

**5**

Animal Statement: The animal is a cat with two extra legs.
Golden Answer: 6
Output: A normal cat has 4 legs.

With two extra legs, the animal has: 4 + 2 = **6**

Animal Statement: The animal is a spider with two extra legs
Golden Answer: 10
Output: A spider normally has 8 legs.

With two extra legs, the animal would have: 8 + 2 = **10**

Animal Statement: The animal is an octopus.
Golden Answer: 8
Output: 0

An octopus has 8 arms, not legs.

Animal Statement: The animal is an octopus that lost two legs and then regrew three legs.
Golden Answer: 9
Output: An octopus normally has 8 legs.

If it lost 2 legs: 8 - 2 = 6 legs

If it then regrew 3 legs: 6 + 3 

ベースラインスコアが得られました！ この初期プロンプトでは正答率が 66.6% でした。

結果をざっと見ると、主に次の2つの問題がありそうです。

### Problem 1: Output formatting issues
ここでの目標は「数値だけ」を出力させることですが、数値以外を含む出力があります：

```
Animal Statement: The animal is a bird.
Golden Answer: 2
Output: Based on the provided animal statement, "The animal is a bird.", the animal has 2 legs.
```

これはプロンプトで改善できそうです。

### Problem 2: Incorrect answers
さらに、答え自体が間違っているケースもあります：

```
Animal Statement: The animal is an octopus that lost two legs and then regrew three legs.
Golden Answer: 9
Output: 5
```

また、次のようなケースもあります：

```
Animal Statement: The animal is a spider with two extra legs
Golden Answer: 10
Output: 8
```

これらは少し「ひっかけ」になっており、モデルがつまずいていそうです。これもプロンプトで改善を試みます。


---

## Our second attempt

初期プロンプトでベースラインが取れたので、プロンプトを改善してスコアが上がるか試してみましょう。

まずは、モデルが数値だけでなく余計な文章も出してしまう問題に取り組みます。以下が2つ目のプロンプト生成関数です：


In [11]:
def build_input_prompt2(animal_statement):
    user_content = f"""You will be provided a statement about an animal and your job is to determine how many legs that animal has.
    
    Here is the animal statement.
    <animal_statement>{animal_statement}</animal_statement>
    
    How many legs does the animal have? Respond only with a numeric digit, like 2 or 6, and nothing else."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

プロンプトの重要な追加点はこの1行です：

> Respond only with a numeric digit, like 2 or 6, and nothing else.


この新しいプロンプトで、各入力をテストします：


In [12]:
outputs2 = [get_completion(build_input_prompt2(question['animal_statement'])) for question in eval_data]

出力をざっと見てみます：


In [13]:
outputs2

['2', '0', '5', '4', '6', '4', '2', '0', '10', '0', '9', '8']

今度は数値だけの出力になりました！

では、ゴールデンアンサーと並べてもう少し詳しく確認します。


In [14]:
for output, question in zip(outputs2, eval_data):
    print(f"Animal Statement: {question['animal_statement']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

Animal Statement: The animal is a human.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a snake.
Golden Answer: 0
Output: 0

Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: 5

Animal Statement: The animal is a dog.
Golden Answer: 4
Output: 4

Animal Statement: The animal is a cat with two extra legs.
Golden Answer: 6
Output: 6

Animal Statement: The animal is an elephant.
Golden Answer: 4
Output: 4

Animal Statement: The animal is a bird.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a fish.
Golden Answer: 0
Output: 0

Animal Statement: The animal is a spider with two extra legs
Golden Answer: 10
Output: 10

Animal Statement: The animal is an octopus.
Golden Answer: 8
Output: 0

Animal Statement: The animal is an octopus that lost two legs and then regrew three legs.
Golden Answer: 9
Output: 9

Animal Statement: The animal is a two-headed, eight-legged mythi

ただし、数値そのものが間違っている問題はまだ残っています。たとえば次のケースです：

```
Animal Statement: The animal is a spider with two extra legs
Golden Answer: 10
Output: 8
```

この問題に取り組む前に、まず公式なスコアを出して、性能が（できれば）改善したかを確認しましょう。


In [15]:
grades = [grade_completion(question['animal_statement'], output, question['golden_answer']) for output, question in zip(outputs2, eval_data)]
print(f"Score: {sum(grades)/len(grades)*100}%")

Animal Statement: The animal is an octopus.
Golden Answer: 8
Output: 0

Score: 91.66666666666666%


スコアは少し上がりました！ **Note: このデータセットは小さいので、結果は参考程度に見てください。**

---


## Our third attempt

次に、次のような誤答で見られる論理的な問題に取り組みます：

```
Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: 6
```

ここで使えるテクニックのひとつが、**chain-of-thought（思考の連鎖）プロンプティング** です。最終回答を出す前に、段階的に推論するよう具体的に指示します。評価が用意できているので、chain-of-thought が本当に効くのかを検証できます。

モデルに `<thinking>` タグ内で「声に出して」考えさせる新しいプロンプトを書きます。ここでロジックが少し複雑になるのは、最終回答を取り出す必要があるためです。そのため、最終回答も `<answer>` タグに入れるよう指示し、数値だけを簡単に抽出できるようにします。


In [16]:
def build_input_prompt3(animal_statement):
    user_content = f"""You will be provided a statement about an animal and your job is to determine how many legs that animal has.
    
    Here is the animal statement.
    <animal_statement>{animal_statement}</animal_statement>
    
    How many legs does the animal have? 
    Start by reasoning about the numbers of legs the animal has, thinking step by step inside of <thinking> tags.  
    Then, output your final answer inside of <answer> tags. 
    Inside the <answer> tags return just the number of legs as an integer and nothing else."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

この新しいプロンプトで出力を収集します：


In [17]:
outputs3 = [get_completion(build_input_prompt3(question['animal_statement'])) for question in eval_data]

いくつかの出力を見てみましょう：


In [18]:
for output, question in zip(outputs3, eval_data):
    print(f"Animal Statement: {question['animal_statement']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

Animal Statement: The animal is a human.
Golden Answer: 2
Output: <thinking>
The animal in question is a human. Let me think about how many legs humans have.

Humans are bipedal mammals, meaning they walk on two legs. A typical human has:
- 2 lower limbs (legs) that they use for walking, standing, and running
- 2 upper limbs (arms) that are not considered legs

So a human has 2 legs.
</thinking>

<answer>2</answer>

Animal Statement: The animal is a snake.
Golden Answer: 0
Output: <thinking>
Let me think about snakes and their legs.

Snakes are reptiles that are characterized by their long, limbless bodies. Through evolution, snakes lost their legs (though some primitive snakes like pythons and boas have small vestigial pelvic bones and spurs, these are not functional legs).

Modern snakes move by using their muscles and scales, not legs. They slither along the ground using their belly scales and body muscles.

So a snake has 0 legs.
</thinking>

<answer>0</answer>

Animal Statement: T

たとえば次のような応答が返ってきます：

```
Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: Here is my step-by-step reasoning:
<thinking>
1. The initial statement says the fox lost a leg.
2. But then the fox "magically grew back the leg he lost and a mysterious extra leg on top of that."
3. This means the fox originally had 4 legs, lost 1 leg, and then grew back the lost leg plus an extra leg, for a total of 5 legs.
</thinking>
<answer>5</answer>
```

少なくともこの例では、論理が改善しているように見えます。

次は、このプロンプトを「採点可能（grade-able）」にすることに注力します。採点前に `<answer>` タグの間の数値を抽出する必要があります。

以下は、`<answer>` タグの間の文字列を取り出す関数です：


In [19]:
import re
def extract_answer(text):
    pattern = r'<answer>(.*?)</answer>'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

では最新の出力群から、最終回答を抽出しましょう：


In [20]:
extracted_outputs3 = [extract_answer(output) for output in outputs3]

In [21]:
extracted_outputs3

['2', '0', '5', '4', '6', '4', '2', '0', '10', None, '9', '8']

次に、スコアを計算して、chain-of-thought を入れたことで違いが出たか確認します！


In [22]:
grades3 = [grade_completion(question['animal_statement'], output, question['golden_answer']) for output, question in zip(extracted_outputs3, eval_data)]
print(f"Score: {sum(grades3)/len(grades3)*100}%")

Animal Statement: The animal is an octopus.
Golden Answer: 8
Output: None

Score: 91.66666666666666%


スコアは 100% まで改善しました！

この評価により、プロンプト変更が実際に出力品質の改善につながっている、という一定の確信が得られます。今回は完全一致（exact-match）採点の単純な例でしたが、次のレッスンではもう少し複雑なケースを扱います。
